In [2]:
# !pip install MTCFeatures

In [3]:
import MTCFeatures
from MTCFeatures import MTCFeatureLoader
import pandas as pd

In [4]:
# MTCFeatures.downloadData(dest='user')

In [5]:
fl = MTCFeatureLoader('MTC-FS-INST-2.0')
seqs = fl.sequences()

In [6]:
%%time
phrase_data = []
for ii, x in enumerate(seqs):
    phrase_data.append({
        'id': x['id'],
        **x['features']
    })

EOFError: Compressed file ended before the end-of-stream marker was reached

In [7]:
df_phrase = pd.DataFrame(phrase_data)
df_phrase.shape # (13492, 62)

(13492, 62)

In [8]:
import numpy as np

In [9]:
# Chaque ligne du set représente une mélodie, représentée par des features.

print(np.array(df_phrase['pitch'][0]).shape) # pitch représente la hauteur des notes
print(np.array(df_phrase['duration'][0]).shape) # duration représente la durée des notes
print(np.array(df_phrase['phrase_end'][0]).shape) # phrase_end dit si la note est la dernière de la phrase

print(df_phrase['pitch'][0])
print(df_phrase['duration'][0])
print(df_phrase['phrase_end'][0])
# On peut donc voir que chacun de ses 3 features est une liste de la même longueur que le nombre de notes dans la mélodie.

# combien de fin de phrase dans le premier exemple ?

print(np.sum(df_phrase['phrase_end'][0])) # 4

# On pense découpere les séquences en sous séquences par notes avec un nombre fixe de notes par sous séquence. On pourra tester de diviser les séquences en sous séquences de nombre 
# fixe de note pour toutes les séquences ou bien de diviser les séquences par un nombre fixe pour toutes les séquences.

print(df_phrase['beat'][0]) # beat représente le temps de la note dans la mesure

# On peut aussi faire des sous séquences basées sur les mesures pour éviter de couper une mesure en deux.

print(df_phrase['beat_fraction_str'][0]) # beat_fraction_str représente le temps de la note dans la mesure sous forme de fraction


(44,)
(44,)
(44,)
['E4', 'C4', 'A3', 'G3', 'C4', 'C4', 'D4', 'C4', 'D4', 'E4', 'A3', 'G3', 'G3', 'F#3', 'G3', 'A3', 'F4', 'E4', 'D4', 'E4', 'D4', 'G3', 'D4', 'E4', 'D4', 'C4', 'A3', 'G3', 'E4', 'F4', 'E4', 'A3', 'E4', 'D4', 'G3', 'A3', 'G3', 'G3', 'F4', 'E4', 'D4', 'A3', 'B3', 'C4']
[1.0, 0.5, 0.5, 0.75, 0.25, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 1.5]
[False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True]
4
[1.0, 2.0, 2.5, 1.0, 1.75, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5,

In [10]:
# print(df_phrase.describe()) # Pas utile pour les features qui sont des listes et trop long

In [11]:
df_phrase.info() # 7302 sequences avec paroles

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13492 entries, 0 to 13491
Data columns (total 62 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     13492 non-null  object
 1   scaledegree            13492 non-null  object
 2   scaledegreespecifier   13492 non-null  object
 3   tonic                  13492 non-null  object
 4   mode                   13492 non-null  object
 5   metriccontour          13492 non-null  object
 6   imaweight              13492 non-null  object
 7   pitch40                13492 non-null  object
 8   midipitch              13492 non-null  object
 9   diatonicpitch          13492 non-null  object
 10  diatonicinterval       13492 non-null  object
 11  chromaticinterval      13492 non-null  object
 12  pitchproximity         13492 non-null  object
 13  pitchreversal          13492 non-null  object
 14  nextisrest             13492 non-null  object
 15  restduration_frac  

In [12]:
for col in df_phrase.columns:
    t = type(df_phrase[col][0])
    if t != list:
        print(col, t)

id <class 'str'>
lyrics <class 'float'>
noncontentword <class 'float'>
wordend <class 'float'>
phoneme <class 'float'>
rhymes <class 'float'>
rhymescontentwords <class 'float'>
wordstress <class 'float'>
melismastate <class 'float'>


7302 séquences avec des paroles

In [13]:
from tqdm import tqdm

In [14]:
def generate_subsequences(sequence, length=8):
    """Découpe une liste en sous-séquences de longueur donnée en ignorant les derniers éléments s'il n'y a pas assez d'éléments"""
    if len(sequence) < length:
        return []
    return [sequence[i:i + length] for i in range(len(sequence) - length + 1)]


In [87]:
i = 50
j = 15

print(df_phrase['midipitch'][i][:])
print(df_phrase['pitch'][i][:])
print(df_phrase['pitch40'][i][:])
print(df_phrase['diatonicpitch'][i][:])

print(df_phrase['scaledegree'][i][:])
print(df_phrase['timesignature'][i][:])



[69, 78, 69, 78, 69, 83, 81, 80, 81, 69, 79, 78, 75, 76, 69, 78, 76, 73, 74, 69, 78, 69, 78, 69, 83, 81, 80, 81, 69, 79, 78, 75, 76, 69, 78, 76, 74, 62, 67, 69, 71, 72, 74, 76, 78, 79, 78, 76, 74, 78, 69, 69, 71, 72, 76, 71, 74, 69, 71, 74, 76, 71, 74, 69, 71, 72, 71, 69, 71, 69, 71, 67, 74, 83, 81, 83, 81, 81, 79, 78, 76, 79, 78, 76, 74, 78, 69, 69, 71, 72, 76, 71, 74, 69, 71, 72, 76, 71, 74, 69, 71, 72, 71, 69, 71, 67, 71, 74, 79, 79]
['A4', 'F#5', 'A4', 'F#5', 'A4', 'B5', 'A5', 'G#5', 'A5', 'A4', 'G5', 'F#5', 'D#5', 'E5', 'A4', 'F#5', 'E5', 'C#5', 'D5', 'A4', 'F#5', 'A4', 'F#5', 'A4', 'B5', 'A5', 'G#5', 'A5', 'A4', 'G5', 'F#5', 'D#5', 'E5', 'A4', 'F#5', 'E5', 'D5', 'D4', 'G4', 'A4', 'B4', 'C5', 'D5', 'E5', 'F#5', 'G5', 'F#5', 'E5', 'D5', 'F#5', 'A4', 'A4', 'B4', 'C5', 'E5', 'B4', 'D5', 'A4', 'B4', 'D5', 'E5', 'B4', 'D5', 'A4', 'B4', 'C5', 'B4', 'A4', 'B4', 'A4', 'B4', 'G4', 'D5', 'B5', 'A5', 'B5', 'A5', 'A5', 'G5', 'F#5', 'E5', 'G5', 'F#5', 'E5', 'D5', 'F#5', 'A4', 'A4', 'B4', 'C5',

on remarque qu'il y a des features qui sont équivalentes comme le `pitch` et le `midipitch`. Ce dernier n'est qu'une représentation du premier en midi, c'est-à-dire en entier. On pourra donc omettre le pitch et ne garder que le midipitch car il est plus simple à manipuler.

on a C5 = 72 et C#5 = 73 car c'est la note juste après C5

On doit chercher toutes les features qui sont corrélées pour ne pas utiliser des features inutiles

[`pitch`, `midipitch`, `pitch40`, `diatonicpitch`] -> `midipitch` : hauteur de la note

[`contour3`, `contour5`, `diatonicinterval`, `chromaticinterval`] -> `chromaticinterval` : variation par rapport à la note précédente

[`tonic`] -> `tonic`

[`mode`] -> `mode`

[`scaledegree`, `scaledegreespecifier`] -> `scaledegree` : peut-être les 2 ?

[`timesignature`] -> `timesignature`




